# Visual Quakes


In [31]:
#Check if Required CSV data exists in Working Directory
#if not present data is downloaded from github repo
#Download CSV data from Github
from pathlib2 import Path

combined = Path("m_combined.csv")
if combined.is_file():
    print("m_combined.csv Opened")
else:
    print("m_combined.csv: DNE -- Downloading from Git")
    url = 'https://raw.githubusercontent.com/nevelo/quake-predict/master/csvData/m_combined.csv'
    fd = wget.download(url)

magCIdat = Path("mag_locationsCI.csv")
if magCIdat.is_file():
    print("mag_locationsCI.csv Opened")
else:
    print("mag_locationsCI.csv: DNE -- Downloading from Git")
    url = 'https://raw.githubusercontent.com/nevelo/quake-predict/master/csvData/mag_locationsCI.csv'
    fd = wget.download(url)
    
magNCdat = Path("mag_locationsNC.csv")
if magNCdat.is_file():
    print("mag_locationsNC.csv Opened")
else:
    print("mag_locationsNC.csv: DNE -- Downloading from Git")
    url = 'https://raw.githubusercontent.com/nevelo/quake-predict/master/csvData/mag_locationsNC.csv'
    fd = wget.download(url)

m_combined.csv Opened
mag_locationsCI.csv Opened
mag_locationsNC.csv Opened


In [21]:
#Open Files
f_combined = open("m_combined.csv")
f_CI = open("mag_locationsCI.csv")
f_NC = open("mag_locationsNC.csv")

In [6]:
#Parse generated .csv file to average out to daily magnitude, power and count of eq > 5.0
if p_avgTable == True:
    manCSV(location)

In [7]:
import urllib
from mpl_toolkits.basemap import Basemap
import numpy as np

# -> http://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php
feed = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/"

# Significant earthquakes in the last 30 days
# url = urllib.request.urlopen(feed + "significant_month.csv")

# Magnitude > 4.5
#url = str(feed + "4.5_month.csv")

# Magnitude > 2.5
url = str(feed + "2.5_month.csv")

# Magnitude > 1.0
# url = str(feed + "1.0_month.csv")

# Reading and storage of data
data = wget.download(url)
data = data.split(b'\n')[+1:-1]
E = np.zeros(len(data), dtype=[('position',  float, 2),
                               ('magnitude', float, 1)])

for i in range(len(data)):
    row = data[i].split(',')
    E['position'][i] = float(row[2]),float(row[1])
    E['magnitude'][i] = float(row[4])

In [8]:
fig = plt.figure(figsize=(14,10))
ax = plt.subplot(1,1,1)

earth = Basemap(projection='mill')


earth.drawcoastlines(color='0.50', linewidth=0.25)
earth.fillcontinents(color='0.95')

In [9]:
P = np.zeros(50, dtype=[('position', float, 2),
                         ('size',     float, 1),
                         ('growth',   float, 1),
                         ('color',    float, 4)])
scat = ax.scatter(P['position'][:,0], P['position'][:,1], P['size'], lw=0.5,
                  edgecolors = P['color'], facecolors='None', zorder=10)

def update(frame):
    current = frame % len(E)
    i = frame % len(P)

    P['color'][:,3] = np.maximum(0, P['color'][:,3] - 1.0/len(P))
    P['size'] += P['growth']

    magnitude = E['magnitude'][current]
    P['position'][i] = earth(*E['position'][current])
    P['size'][i] = 5
    P['growth'][i]= np.exp(magnitude) * 0.1

    if magnitude < 6:
        P['color'][i]    = 0,0,1,1
    else:
        P['color'][i]    = 1,0,0,1
    scat.set_edgecolors(P['color'])
    scat.set_facecolors(P['color']*(1,1,1,0.25))
    scat.set_sizes(P['size'])
    scat.set_offsets(P['position'])
    return scat,


animation = animation.FuncAnimation(fig, update, interval=10)
plt.show()


ZeroDivisionError: integer division or modulo by zero